In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


# Import Data
data = pd.read_csv("/kaggle/input/santa-2024/sample_submission.csv")
# print(data) # Correctly locates competition data
text_data = data['text'].tolist()

   id                                               text
0   0  advent chimney elf family fireplace gingerbrea...
1   1  advent chimney elf family fireplace gingerbrea...
2   2  yuletide decorations gifts cheer holiday carol...
3   3  yuletide decorations gifts cheer holiday carol...
4   4  hohoho candle poinsettia snowglobe peppermint ...
5   5  advent chimney elf family fireplace gingerbrea...


In [ ]:
from nltk import bigrams
from collections import Counter, defaultdict
from nltk.tokenize.treebank import TreebankWordDetokenizer
import nltk

# Tokenize data to compare tokens
tokenized_data = []
for sentence in text_data:
    tokenized_data += nltk.word_tokenize(sentence)
    
# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

for idx, row in data.iterrows():
    words = row['text'].lower().split()
    for w1, w2 in bigrams(words):
        model[w1][w2] += 1

# Calculate probability of w2 if previously at w1
for w1 in model:
    total_count = float(sum(model[w1].values()))
    for w2 in model[w1]:
        model[w1][w2] /= total_count

def calculate_sequence_probability(sequence):
    if len(sequence) < 2:
        return 0
    
    prob = 1 # Default 
    words = sequence.lower().split()
    for w1, w2 in bigrams(words):
        prob *= (model[w1][w2] + 1e-10)  # Add smoothing to avoid zero probabilities
    return prob

# Idea Implementation:
# Create a loop to compare each word one by one as a pair with every other word in the row for each row in the csv
# Establish a min probability of 1 (just for comparison sake)
# For each word in the list besides current word, calculate bigram_probability
# If bigram_probability of this pair is less than min_probability of all pairs, set min_probability to this and set this word after our current word
# Account for moving arounds values each time, and repeat

result_df = data.copy()
detokenizer = TreebankWordDetokenizer()
for idx, row in data.iterrows():
    words = row['text'].split()
    best_sequence = words.copy()
    best_prob = calculate_sequence_probability(' '.join(best_sequence))

    # Simple greedy optimization
    sort = True
    while sort:
        sort = False
        for i in range(len(words)):
            for j in range(i+1, len(words)):
                # swap our current word with each word in the list
                test_sequence = best_sequence.copy()
                test_sequence[i], test_sequence[j] = test_sequence[j], test_sequence[i]

                # calculate probability of each specific pair, keep if better
                test_prob = calculate_sequence_probability(' '.join(test_sequence))

                if test_prob > best_prob:
                    best_sequence = test_sequence
                    best_prob = test_prob
                    improved = true

    
    # and then put data text row into the same order as the best order, provided via the sorted probabilities dictionary
    result_df.loc[idx, 'text'] = ' '.join(best_sequence)

    # Put back into correct row of text column in given csv file for submission
result_df.to_csv('submission.csv', index=False)

        
        
    
    